# Lab 5: Measurement Uncertainty
### MAE 3120, Spring 2022

**Jade Greenberg & Caden Gobat**

**Background:** Very few (if any) observations or measurements can be known and reported with absolute certainty. Because of this, understanding and describing the uncertainties that are inherent to our measurements is an integral part of working with experimental data. At its core, reporting an uncertainty value is a statement of probability. Stating an error of $\pm u$ is a claim (with some percent certainty, which varies based on the desired confidence level) that the true value falls within $u$ of what was measured. In this lab, we assess and analyze several different sources of uncertainty in the measurement of the resistance value of a resistor with a nominal value of $22\ \Omega$.

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from scipy import stats

## Part I: Type A Uncertainty

In [ ]:
bias = 0.01 # residual error from touching leads
measurements = np.array([21.739,21.742,21.741,21.738,21.735,21.733,21.736,21.735,21.734,21.731]) # 10 measurements
measurements -= bias # correct residual resistance
N = len(measurements) # number of trials
u1 = stats.sem(measurements) # standard error in the mean
print(f"Average measurement ({N} trials): {measurements.mean():.3f} ± {u1:.3f} Ω")

Average measurement (10 trials): 21.726 ± 0.001 Ω


This statistic arises from the mean of $N$ samples, so there are $N-1$ degrees of freedom. Here we have 10 samples, so 9 degrees of freedom (dof). This error is attributed to statistical randomness in the physical processes, so with enough trials, it should follow a Gaussian/normal distribution. With only $N=10$, we use a Student-$t$ distribution to model it.

## Part II: Type B Uncertainty

### Range and reading standard uncertainty, $u_2$

1) What is the range the instrument is set at to measure a 22 Ω resistance? In Appendix A, find and report the proper values for the range and reading uncertainties.
> The range the instrument defaults to for the $22\ \Omega$ resistor is $100\ \Omega$. Therefore the uncertainty from the datasheet is $\pm(0.05\text{\% of the reading} + 0.008\text{\% of the range})$

2) In the DMM datasheet, what does the “math null enable” (footnote [3]) mean? In the future you could use it to automatically correct for the residual resistance.
> This is essentially a toggle to tell the instrument whether or not to perform an inline bias correction based on its own measurement of the "null" reading.

3) From this, calculate the measurement uncertainty due to these two sources at 68% confidence level.

> We have $a=\pm(0.0005\bar{R}+0.0008\cdot100)\ \Omega$ and $\bar{R}\cong 21.726\ \Omega$. Therefore $a\approx\pm0.0189\ \Omega \implies u_3=\frac{a}{\sqrt{3}}=\frac{\pm0.0189}{\sqrt{3}}=\pm0.0109\ \Omega$.

4) What is the number of degrees of freedom for this uncertainty? While for Type A uncertainties this was easy to estimate, for Type B, it is often considered as infinite, such as for specifications reported by the manufacturer.

> We will assume the degrees of freedom are infinite, since no furhter information is given in the datasheet.

5) What is the type of probability distribution function?

> The most conservative assumption will be to treat this as a rectangular distribution, where all values within the range are equally likely.

In [ ]:
DMM_err = lambda R : 0.0005*R + 0.00008*100
u2 = DMM_err(measurements.mean())/np.sqrt(3)
u2

0.010890673597777747

### DMM resolution, $u_3$

1) Measurements include errors due to limited resolution of the display panel. This is usually taken as one half of the least significant digit, LSD. Remember, however, the standard uncertainties must be expressed at $\pm1\sigma$, i.e. you need to know the probability distribution function of this uncertainty.

2) What kind of pdf should you assume this uncertainty follow?
> This should be a uniform (rectangular) distribution, with a width $2a$ corresponding to 100% confidence.

3) Calculate the standard resolution uncertainty.
> Our instrument has 3 digits of precision after the decimal place. Thus one-half the LSD is $\pm0.0005\ \Omega$. Therefore $u_3=\frac{\pm0.0005}{\sqrt{3}}\approx\pm0.0003\ \Omega$.

4) What is the number of degrees of freedom for this uncertainty?

> It is only dependent on one thing: the precision/number of display digits of the multimeter, which does not even depend on how many digits of precision the multimeter actually *has*—it is still 1.

In [ ]:
a3 = 0.0005
u3 = a3/np.sqrt(3)
u3

0.0002886751345948129

## Part III: Combined Standard Uncertainty

| Source of Uncertainty |   Type   | $u_i$ | Uncertainty value (Unit) | Sensitivity coefficient ($c_i$) | Type of pdf | Standard uncertainty (unit) | Degrees of freedom ($\nu_i$) |
|:---------------------:|:--------:|:-----:|:------------------------:|:-------------------------------:|:-----------:|:---------------------------:|:--------------------------:|
|     **Repeatability**     |     A    | $u_1$ | $\pm0.001\ \Omega$ |1|Student-$t$|$\pm0.001\ \Omega$|9|
|   **Range and Reading**   |     B    | $u_2$ | $\pm0.0189\ \Omega$ |1| Assumed uniform | $\pm0.0109\ \Omega$ | $\infty$ |
|       **Resolution**      |     B    | $u_3$ | $\pm0.0005\ \Omega$ |1|Uniform|$\pm0.0003\ \Omega$| 1 |
|  **Current Measurement**  | Combined | $u_c$ |             -            |                -                | Converges to normal | $\pm0.011\ \Omega$| ~80,000 |

<br>
Welch-Satterthwaite formula: 

$$ \nu_\text{eff} = \frac{u_c^4}{\Sigma_{i=1}^N \large \frac{c_i^4 u_i^4}{\nu_i}} $$

In [ ]:
u = np.array([u1,u2,u3]) # array of individual error contributions
uc = np.sqrt(np.sum(u**2)) # combined standard uncertainty
v_eff = lambda uc, ci, ui, vi : uc**4 / np.sum(ci**4 * ui**4 / vi) # Welch-Satterthwaite
c = np.array([1,1,1])
v = np.array([9,np.inf,1])
print("Combined uncertainty:",uc)
print("Effective dof:",v_eff(uc,c,u,v))

Combined uncertainty: 0.010951666249277108
Effective dof: 79809.14047748108


## Part IV: Expanded Uncertainty

1) In light of your effective degrees of freedom, which pdf are you planning on following to estimate you coverage factor $k$? If you had an infinite effective degrees of freedom, which pdf would you follow?

> Since $\nu_\text{eff}$ is so high, we can proceed using a normal distribution/Gaussian PDF. This is also due to the central limit theorem, from which we know that when we combine several distinct sources of uncertainty, their aggregate PDF converges to a single normal distribution.

2) What is the value of $k$?

> For 95\% confidence, $k\approx1.96$. This makes sense because we know that $2\sigma$ uncertainty has approximately 95.45\% coverage, and in fact, $k=2$ is often used to approximate 95\% confidence.

3) What is your expanded uncertainty, $U_m$? Use the table you completed earlier for your overall uncertainty budget.

> The combined standard uncertainty was $u_c = \pm 0.011\ \Omega$. To get the expanded uncertainty at the desired confidence lebel, we multiply by the coverage factor, i.e. $U_m = k\cdot u_c = 1.96\cdot 0.011\ \Omega \approx 0.022\ \Omega$.

4) Finally, your work is not complete until you report the final result in the form $R=\bar{R}\pm U_m\ \Omega$.

> We report $R=21.736\pm0.022\ \Omega$ with 95\% confidence ($2\sigma$ uncertainty).

## Discussion

1. Often the repeatability uncertainty you calculated in section 1, is very small and only Type B uncertainties are adequate to characterize the overall uncertainty. Is this the case here? In case it is, why is this important to your tests?

> Yes, $u_2$ dominates the final uncertainty. The implication is that the instrument is more *precise* than it necessarily is *accurate*.

2. How can you estimate environment effects (i.e. change in temperature, pressure) on your instruments and subsequently on your measurements? What type of uncertainties are they?

> This is possible through calibration and testing in a variety of known/controlled environmental settings. These errors would be classified as Type B, since they are not statistical, but rather the result of physical effects.

3. Prove analytically that the coverage factor for the Least Significant Digit uncertainty (resolution limitation) is $\frac{1}{\sqrt{3}}$. You will need the following information: the actual value below the LSD is equiprobable, i.e. it follows a rectangular distribution function.

> A uniform/rectangular pdf is described by $\displaystyle p(x) = \begin{cases} \frac{1}{b-a} & x \in (a,b) \\ 0 & \text{otherwise}\end{cases}$

> The error on a measurement $x$ is given by $\displaystyle E(x) = \int_a^b x\cdot p(x)\ dx = \int_a^b \frac{x}{b-a} dx = \frac{a+b}{2}$. Therefore, $\displaystyle \sigma^2 = \int_a^b \left(x-E(x)\right)^2p(x)\ dx = \int_a^b \left(x-\frac{a+b}{2}\right)^2 \frac{1}{b-a}dx = \frac{1}{3}\left(\frac{b-a}{2}\right)^2$. Taking the square root of both sides shows that the standard error $\sigma$ is given by the half-width of the distribution (i.e. $(b-a)/2$) divided by $\sqrt{3}$.

4. Is the DMM display resolution important compared to the other sources of uncertainties?
    > No, it is nearly an order of magnitude smaller than the statistical error from the standard deviation of the measurements, which is itself an order of magnitude smaller than the range/reading error of the instrument. When added in quadrature, only the largest terms really matter.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=385eace9-e066-4866-bb55-3a1ff5bdb476' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>